In [1]:
import torch

In [2]:
img_t = torch.randn(3, 5, 5) #[channels, rows, cols]
weights = torch.tensor([0.2126, 0.7152, 0.0722])
batch_t = torch.rand(2, 3, 5, 5) #[batch, channels, rows, cols]

In [3]:
img_gray_naive = img_t.mean(-3)
batch_gray_naive = batch_t.mean(-3)
print("img_gray_naive shape: {}\nbatch_gray_naive shape: {}".format(
    img_gray_naive.shape, 
    batch_gray_naive.shape
))

img_gray_naive shape: torch.Size([5, 5])
batch_gray_naive shape: torch.Size([2, 5, 5])


In [4]:
unsqueezed_weights = weights.unsqueeze(-1).unsqueeze(-1)
img_weights = img_t * unsqueezed_weights
batch_weights = batch_t * unsqueezed_weights
img_gray_weighted = img_weights.sum(-3)
batch_gray_weighted = batch_weights.sum(-3)
print("img_gray_weighted shape: {}\nbatch_gray_weighted shape: {}".format(
    img_gray_weighted.shape,
    batch_gray_weighted.shape
))

img_gray_weighted shape: torch.Size([5, 5])
batch_gray_weighted shape: torch.Size([2, 5, 5])


### Named tensors

In [5]:
weighted_named = torch.tensor([0.2126, 0.7152, 0.0722], names = ['channels'])
print(weighted_named)

tensor([0.2126, 0.7152, 0.0722], names=('channels',))


/tmp/pip-req-build-sqyrlut3/c10/core/TensorImpl.h:845: UserWarning: Named tensors and all their associated APIs are an experimental feature and subject to change. Please do not use them for anything important until they are released as stable.


In [6]:
img_named = img_t.refine_names(..., 'channels', 'rows', 'cols')
batch_named = batch_t.refine_names(..., 'channels', 'rows', 'cols')
print("img named:", img_named.shape, img_named.names)
print("batch named:", batch_named.shape, batch_named.names)

img named: torch.Size([3, 5, 5]) ('channels', 'rows', 'cols')
batch named: torch.Size([2, 3, 5, 5]) (None, 'channels', 'rows', 'cols')


In [7]:
weights_aligned = weighted_named.align_as(img_named)
weights_aligned.shape, weights_aligned.names

(torch.Size([3, 1, 1]), ('channels', 'rows', 'cols'))

In [8]:
gray_named = (img_named * weights_aligned).sum('channels')
gray_named.shape, gray_named.names

(torch.Size([5, 5]), ('rows', 'cols'))

In [9]:
gray_plain = gray_named.rename(None)
gray_plain.shape, gray_plain.names

(torch.Size([5, 5]), (None, None))

### Tensor dtype

In [10]:
double_points = torch.ones(10, 2, dtype = torch.double)
short_points = torch.tensor([[1, 2], [3, 4]], dtype = torch.short)
print(double_points.dtype)
print(short_points.dtype)

torch.float64
torch.int16


In [11]:
double_point = torch.ones(10, 2).double()
short_point = torch.tensor([[1, 2], [3, 4]]).short()
print(double_points.dtype)
print(short_points.dtype)

torch.float64
torch.int16


In [12]:
double_point = torch.ones(10, 2).to(torch.double)
short_point = torch.zeros(10, 2).to(torch.short)
print(double_points.dtype)
print(short_points.dtype)

torch.float64
torch.int16


### Tensor APIs

In [13]:
a = torch.ones(3, 2)
a_t = torch.transpose(a, 0, 1)
print("a.shape = {}, a_t.shape = {}".format(a.shape, a_t.shape))

a.shape = torch.Size([3, 2]), a_t.shape = torch.Size([2, 3])


In [14]:
a = torch.ones(3, 2)
a_t = a.transpose(0, 1)
print("a.shape = {}, a_t.shape = {}".format(a.shape, a_t.shape))

a.shape = torch.Size([3, 2]), a_t.shape = torch.Size([2, 3])


### Storage of a tensor

In [15]:
points = torch.tensor([[4.0, 1.0], [5.0, 3.0], [2.0, 1.0]])
print(points.storage())
point_storage = points.storage()
print(point_storage[0])
print(point_storage[1])
point_storage[0] = 2.0
print(points)

 4.0
 1.0
 5.0
 3.0
 2.0
 1.0
[torch.FloatStorage of size 6]
4.0
1.0
tensor([[2., 1.],
        [5., 3.],
        [2., 1.]])


In [16]:
a = torch.ones(3, 2)
a.zero_()
print(a)

tensor([[0., 0.],
        [0., 0.],
        [0., 0.]])


In [17]:
second_point = points[1]
print(second_point.storage_offset())
print(second_point.size())
print(second_point.shape)
print(second_point.storage_offset())
print(second_point.stride())
print(points.stride())

2
torch.Size([2])
torch.Size([2])
2
(1,)
(2, 1)


In [18]:
second_point = points[1].clone()
second_point[0] = 10.0
print(points)

tensor([[2., 1.],
        [5., 3.],
        [2., 1.]])


### Transpose without copying

In [19]:
points_t = points.t()
print(id(points.storage()) == id(points_t.storage()))
print(points.stride())
print(points_t.stride())

True
(2, 1)
(1, 2)


### Transpose in higher dimension

In [20]:
some_t = torch.ones([3, 4, 5])
transpose_t = some_t.transpose(0, 2)
print(some_t.shape)
print(transpose_t.shape)
print(some_t.stride())
print(transpose_t.stride())

torch.Size([3, 4, 5])
torch.Size([5, 4, 3])
(20, 5, 1)
(1, 5, 20)


### Contiguous tensor

In [21]:
print(points.is_contiguous())
print(points_t.is_contiguous())

True
False


In [22]:
points = torch.tensor([[4.0, 1.0], [5.0, 3.0], [2.0, 1.0]])
points_t = points.t()

print(points_t)
print(points_t.storage())
print(points.stride())
print(points_t.stride())

points_t_cont = points_t.contiguous()
print(points_t_cont)
print(points_t_cont.stride())
print(points_t_cont.storage())

tensor([[4., 5., 2.],
        [1., 3., 1.]])
 4.0
 1.0
 5.0
 3.0
 2.0
 1.0
[torch.FloatStorage of size 6]
(2, 1)
(1, 2)
tensor([[4., 5., 2.],
        [1., 3., 1.]])
(3, 1)
 4.0
 5.0
 2.0
 1.0
 3.0
 1.0
[torch.FloatStorage of size 6]


### Torch Tensor <-> Numpy

In [23]:
points = torch.ones(3, 4)
points_np = points.numpy()
print(points_np)

[[1. 1. 1. 1.]
 [1. 1. 1. 1.]
 [1. 1. 1. 1.]]


In [24]:
points = torch.from_numpy(points_np)
print(points)

tensor([[1., 1., 1., 1.],
        [1., 1., 1., 1.],
        [1., 1., 1., 1.]])


### Serialize Tensor

In [25]:
torch.save(points, "./points1.t")

In [26]:
with open("./points2.t", "wb") as f:
    torch.save(points, f)

In [27]:
points1 = torch.load("./points1.t")
print(points1)

tensor([[1., 1., 1., 1.],
        [1., 1., 1., 1.],
        [1., 1., 1., 1.]])


In [28]:
with open("./points2.t", "rb") as f:
    points2 = torch.load(f)
    print(points2)

tensor([[1., 1., 1., 1.],
        [1., 1., 1., 1.],
        [1., 1., 1., 1.]])


In [29]:
import h5py

In [30]:
f = h5py.File("./points.hdf5", "w")
dset = f.create_dataset('coords', data = points.numpy())
f.close

<bound method File.close of <HDF5 file "points.hdf5" (mode r+)>>

In [31]:
f = h5py.File("./points.hdf5", "r")
dset = f['coords']
last_point = dset[-2:]
print(last_point)

[[1. 1. 1. 1.]
 [1. 1. 1. 1.]]
